In [84]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/model/keras/default/1/brats_segmentation_model.h5
/kaggle/input/train-model/tensorflow2/default/1/brats_segmentation_model.h5
/kaggle/input/instant-odc-ai-hackathon/sample_submission.csv
/kaggle/input/instant-odc-ai-hackathon/test/preprocess.ipynb
/kaggle/input/instant-odc-ai-hackathon/test/BraTS2021_01358/BraTS2021_01358_flair.nii
/kaggle/input/instant-odc-ai-hackathon/test/BraTS2021_01358/BraTS2021_01358_t2.nii
/kaggle/input/instant-odc-ai-hackathon/test/BraTS2021_01358/BraTS2021_01358_t1ce.nii
/kaggle/input/instant-odc-ai-hackathon/test/BraTS2021_01358/BraTS2021_01358_t1.nii
/kaggle/input/instant-odc-ai-hackathon/test/BraTS2021_01435/BraTS2021_01435_t1ce.nii
/kaggle/input/instant-odc-ai-hackathon/test/BraTS2021_01435/BraTS2021_01435_flair.nii
/kaggle/input/instant-odc-ai-hackathon/test/BraTS2021_01435/BraTS2021_01435_t2.nii
/kaggle/input/instant-odc-ai-hackathon/test/BraTS2021_01435/BraTS2021_01435_t1.nii
/kaggle/input/instant-odc-ai-hackathon/test/BraTS2021_01398/BraT

In [85]:
# MRI & NIfTI
import nibabel as nib
import cv2  # for resizing slices

# Deep Learning
import tensorflow as tf
from tensorflow.keras import layers, models

# Data splitting
from sklearn.model_selection import train_test_split

# Visualization
import matplotlib.pyplot as plt

In [86]:
DATA_PATH = '/kaggle/input/instant-odc-ai-hackathon'

TRAIN_PATH = os.path.join(DATA_PATH, "Train")
TEST_PATH  = os.path.join(DATA_PATH, "test")

In [87]:
def load_train_records(path):
    patient_records = []
    patients = sorted(os.listdir(path))

    for patient in patients:
        p_path = os.path.join(path, patient)
        if not os.path.isdir(p_path):
            continue

        files = os.listdir(p_path)

        flair = [f for f in files if f.lower().endswith('_flair.nii')]
        t1    = [f for f in files if f.lower().endswith('_t1.nii')]
        t1ce  = [f for f in files if f.lower().endswith('_t1ce.nii')]
        t2    = [f for f in files if f.lower().endswith('_t2.nii')]
        seg   = [f for f in files if f.lower().endswith('_seg.nii')]

        if not (flair and t1 and t1ce and t2 and seg):
            print(f"Skipping incomplete patient: {patient}")
            continue

        paths = {
            "flair": os.path.join(p_path, flair[0]),
            "t1":    os.path.join(p_path, t1[0]),
            "t1ce":  os.path.join(p_path, t1ce[0]),
            "t2":    os.path.join(p_path, t2[0]),
            "seg":   os.path.join(p_path, seg[0]),
        }

        # ✅ Check empty or corrupted files
        try:
            for p in paths.values():
                if os.path.getsize(p) == 0:
                    raise ValueError("Empty file")
                nib.load(p)  # test load
        except Exception as e:
            print(f"Skipping corrupted patient {patient}: {e}")
            continue

        patient_records.append(paths)

    return patient_records


In [88]:
def load_test_records(path):
    test_records = []
    patients = sorted(os.listdir(path))

    for patient in patients:
        p_path = os.path.join(path, patient)
        if not os.path.isdir(p_path):
            continue

        files = os.listdir(p_path)

        flair = [f for f in files if f.lower().endswith('_flair.nii')]
        t1    = [f for f in files if f.lower().endswith('_t1.nii')]
        t1ce  = [f for f in files if f.lower().endswith('_t1ce.nii')]
        t2    = [f for f in files if f.lower().endswith('_t2.nii')]

        if not (flair and t1 and t1ce and t2):
            print(f"Skipping incomplete test patient: {patient}")
            continue

        paths = {
            "flair": os.path.join(p_path, flair[0]),
            "t1":    os.path.join(p_path, t1[0]),
            "t1ce":  os.path.join(p_path, t1ce[0]),
            "t2":    os.path.join(p_path, t2[0]),
        }

        try:
            for p in paths.values():
                if os.path.getsize(p) == 0:
                    raise ValueError("Empty file")
                nib.load(p)
        except Exception as e:
            print(f"Skipping corrupted test patient {patient}: {e}")
            continue

        test_records.append(paths)

    return test_records


In [89]:
train_records = load_train_records(TRAIN_PATH)
test_records  = load_test_records(TEST_PATH)

print("Train:", len(train_records))
print("Test:", len(test_records))


Skipping corrupted patient BraTS2021_00000: Empty file
Skipping corrupted patient BraTS2021_00002: Empty file
Skipping corrupted patient BraTS2021_00003: Empty file
Skipping corrupted patient BraTS2021_00005: Empty file
Skipping corrupted patient BraTS2021_00006: Empty file
Skipping corrupted patient BraTS2021_00008: Empty file
Skipping corrupted patient BraTS2021_00009: Empty file
Skipping corrupted patient BraTS2021_00011: Empty file
Skipping corrupted patient BraTS2021_00012: Empty file
Skipping corrupted patient BraTS2021_00014: Empty file
Skipping corrupted patient BraTS2021_00016: Empty file
Skipping corrupted patient BraTS2021_00017: Empty file
Skipping corrupted patient BraTS2021_00018: Empty file
Skipping corrupted patient BraTS2021_00019: Empty file
Skipping corrupted patient BraTS2021_00020: Empty file
Skipping corrupted patient BraTS2021_00021: Empty file
Skipping corrupted patient BraTS2021_00022: Empty file
Skipping corrupted patient BraTS2021_00024: Empty file
Skipping c

In [90]:
train_rec, val_rec = train_test_split(
    train_records,
    test_size=0.2,
    random_state=42
)

print("Train:", len(train_rec))
print("Validation:", len(val_rec))

Train: 436
Validation: 109


In [95]:
train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator_sparse(train_rec, batch_size=8),
    output_types=(tf.float32, tf.uint8),
    output_shapes=([None,128,128,2],[None,128,128])
).prefetch(tf.data.AUTOTUNE)


In [98]:
def normalize_slice(x):
    """
    Safely normalize a single MRI slice.
    """
    mean = x.mean()
    std = x.std()
    if std < 1e-6:
        std = 1.0
    return (x - mean) / std


In [99]:
def preprocess_slice(flair_slice, t1ce_slice, seg_slice=None, label_map=None, augment=False):
    # Resize to 128x128
    flair_slice = cv2.resize(flair_slice, (128,128))
    t1ce_slice  = cv2.resize(t1ce_slice,  (128,128))

    # Normalize
    flair_slice = normalize_slice(flair_slice)
    t1ce_slice  = normalize_slice(t1ce_slice)

    x = np.stack([flair_slice, t1ce_slice], axis=-1)

    if seg_slice is not None:
        y = np.vectorize(lambda v: label_map.get(v,0))(seg_slice)
        y = cv2.resize(y.astype(np.uint8), (128,128), interpolation=cv2.INTER_NEAREST)
    else:
        y = None

    # Augmentation
    if augment:
        if np.random.rand() < 0.5:
            x = np.flip(x, axis=1)  # horizontal flip
            if y is not None:
                y = np.fliplr(y)
        if np.random.rand() < 0.5:
            x = np.flip(x, axis=0)  # vertical flip
            if y is not None:
                y = np.flipud(y)

    return x, y


In [100]:
def tf_data_generator(records, batch_size=8, augment=True):
    label_map = {0:0, 1:1, 2:2, 4:3}  # BraTS labels -> 0-3

    def generator():
        x_batch, y_batch = [], []
        while True:
            for rec in records:
                try:
                    flair_vol = nib.load(rec["flair"]).get_fdata()
                    t1ce_vol  = nib.load(rec["t1ce"]).get_fdata()
                    seg_vol   = nib.load(rec["seg"]).get_fdata()
                except:
                    continue

                D = flair_vol.shape[2]
                for i in range(D):
                    x, y = preprocess_slice(flair_vol[:,:,i], t1ce_vol[:,:,i], seg_vol[:,:,i], label_map, augment)
                    if np.sum(y) == 0 and np.random.rand() > 0.3:
                        continue

                    x_batch.append(x)
                    y_batch.append(y)

                    if len(x_batch) == batch_size:
                        yield np.array(x_batch, dtype=np.float32), np.array(y_batch, dtype=np.uint8)
                        x_batch, y_batch = [], []

    return tf.data.Dataset.from_generator(
        generator,
        output_types=(tf.float32, tf.uint8),
        output_shapes=([batch_size,128,128,2],[batch_size,128,128])
    ).prefetch(tf.data.AUTOTUNE)


In [101]:
def tf_test_generator(records):
    for rec in records:
        try:
            flair_vol = nib.load(rec["flair"]).get_fdata()
            t1ce_vol  = nib.load(rec["t1ce"]).get_fdata()
        except Exception as e:
            print("Skipping corrupted test case:", e)
            continue

        H, W, D = flair_vol.shape
        slices = []

        for i in range(D):
            x, _ = preprocess_slice(flair_vol[:,:,i], t1ce_vol[:,:,i], seg_slice=None, augment=False)
            slices.append(x)

        yield np.array(slices, dtype=np.float32), rec, (H, W, D)


In [102]:
def unet(input_shape=(128,128,2)):
    inp = layers.Input(input_shape)
    
    c1 = layers.Conv2D(16,3,activation='relu',padding='same')(inp)
    c1 = layers.Conv2D(16,3,activation='relu',padding='same')(c1)
    p1 = layers.MaxPooling2D()(c1)

    c2 = layers.Conv2D(32,3,activation='relu',padding='same')(p1)
    c2 = layers.Conv2D(32,3,activation='relu',padding='same')(c2)
    p2 = layers.MaxPooling2D()(c2)

    c3 = layers.Conv2D(64,3,activation='relu',padding='same')(p2)
    c3 = layers.Conv2D(64,3,activation='relu',padding='same')(c3)
    p3 = layers.MaxPooling2D()(c3)

    b = layers.Conv2D(128,3,activation='relu',padding='same')(p3)
    b = layers.Conv2D(128,3,activation='relu',padding='same')(b)

    u3 = layers.UpSampling2D()(b)
    u3 = layers.Concatenate()([u3,c3])
    c4 = layers.Conv2D(64,3,activation='relu',padding='same')(u3)
    c4 = layers.Conv2D(64,3,activation='relu',padding='same')(c4)

    u2 = layers.UpSampling2D()(c4)
    u2 = layers.Concatenate()([u2,c2])
    c5 = layers.Conv2D(32,3,activation='relu',padding='same')(u2)
    c5 = layers.Conv2D(32,3,activation='relu',padding='same')(c5)

    u1 = layers.UpSampling2D()(c5)
    u1 = layers.Concatenate()([u1,c1])
    c6 = layers.Conv2D(16,3,activation='relu',padding='same')(u1)
    c6 = layers.Conv2D(16,3,activation='relu',padding='same')(c6)

    out = layers.Conv2D(4,1,activation='softmax')(c6)  # 4 classes
    return models.Model(inp, out)

model = unet()

In [103]:
def dice_metric_multiclass(y_true, y_pred, smooth=1e-6):
    """
    Computes mean Dice coefficient for multi-class segmentation.
    y_true: integer labels (batch,H,W)
    y_pred: softmax (batch,H,W,C)
    """
    y_true_onehot = tf.one_hot(tf.cast(y_true, tf.int32), depth=4)
    y_pred = tf.cast(y_pred, tf.float32)

    axes = (1,2)  # sum over H,W
    intersection = tf.reduce_sum(y_true_onehot * y_pred, axis=axes)
    union = tf.reduce_sum(y_true_onehot + y_pred, axis=axes)
    dice = (2*intersection + smooth) / (union + smooth)
    return tf.reduce_mean(dice)

In [104]:
# train_gen = data_generator_sparse(train_rec, batch_size=8)
# val_gen   = data_generator_sparse(val_rec, batch_size=8)

# model.compile(
#     optimizer=tf.keras.optimizers.Adam(1e-7),
#     loss='sparse_categorical_crossentropy',
#     metrics=[dice_metric_multiclass]
# )

# history = model.fit(
#     train_gen,
#     steps_per_epoch=len(train_rec)*20 // 8,
#     validation_data=val_gen,
#     validation_steps=len(val_rec)*20 // 8,
#     epochs=30
# )

In [112]:
train_dataset = tf_data_generator(train_rec, batch_size=8, augment=True)
val_dataset   = tf_data_generator(val_rec,   batch_size=8, augment=False)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=[dice_metric_multiclass]
)

history = model.fit(
    train_dataset,
    steps_per_epoch=500,   # start small for testing
    validation_data=val_dataset,
    validation_steps=20,   # adjust as needed
    epochs=15
)


Epoch 1/15
600/600 ━━━━━━━━━━━━━━━━━━━━ 118s 185ms/step - dice_metric_multiclass: 0.2557 - loss: 0.0816 - val_dice_metric_multiclass: 0.2603 - val_loss: 0.0692
Epoch 2/15
600/600 ━━━━━━━━━━━━━━━━━━━━ 110s 184ms/step - dice_metric_multiclass: 0.2628 - loss: 0.0636 - val_dice_metric_multiclass: 0.2706 - val_loss: 0.0587
Epoch 3/15
600/600 ━━━━━━━━━━━━━━━━━━━━ 112s 187ms/step - dice_metric_multiclass: 0.2715 - loss: 0.0607 - val_dice_metric_multiclass: 0.2802 - val_loss: 0.0511
Epoch 4/15
600/600 ━━━━━━━━━━━━━━━━━━━━ 115s 192ms/step - dice_metric_multiclass: 0.2775 - loss: 0.0540 - val_dice_metric_multiclass: 0.2882 - val_loss: 0.0477
Epoch 5/15
600/600 ━━━━━━━━━━━━━━━━━━━━ 130s 218ms/step - dice_metric_multiclass: 0.2978 - loss: 0.0606 - val_dice_metric_multiclass: 0.3076 - val_loss: 0.0431
Epoch 6/15
600/600 ━━━━━━━━━━━━━━━━━━━━ 129s 216ms/step - dice_metric_multiclass: 0.3153 - loss: 0.0571 - val_dice_metric_multiclass: 0.3038 - val_loss: 0.0573
Epoch 7/15
600/600 ━━━━━━━━━━━━━━━━━━━━ 

In [113]:
def rle_encode(mask):
    """
    Convert 2D integer mask (H,W) to RLE.
    mask: 2D numpy array with values 0-3
    Returns RLE string per class.
    """
    rle_all = []

    for c in range(1, 4):  # skip background 0
        m = (mask == c).astype(np.uint8)
        pixels = m.flatten(order='F')
        pixels = np.concatenate([[0], pixels, [0]])
        runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
        runs[1::2] -= runs[::2]
        rle_all.append(' '.join(str(x) for x in runs) if runs.size > 0 else "")
    return rle_all  # list of 3 strings, one per class


In [114]:
def predict_patient(model, patient_record):
    """
    Predicts the segmentation mask for a full patient volume.
    Returns mask of shape (H,W,D) with integer labels 0-3
    """
    slices, rec, (H, W, D) = next(tf_test_generator([patient_record]))
    preds = model.predict(slices, verbose=0)  # (num_slices,128,128,4)
    preds_class = np.argmax(preds, axis=-1)   # integer labels 0-3
    # resize back to original HxW
    mask_vol = np.zeros((H, W, D), dtype=np.uint8)
    for i in range(D):
        mask_vol[:,:,i] = cv2.resize(preds_class[i], (W, H), interpolation=cv2.INTER_NEAREST)
    return mask_vol


In [ ]:
submission = []

for rec in test_records:
    slices, record, shape = next(test_generator([rec]))
    preds = model.predict(slices)
    preds_class = np.argmax(preds, axis=-1)  # shape (H, W, D)

    patient_id = record.get("id") or record.get("flair").split('/')[-2]

    # BraTS expects 3 rows per patient (classes 1,2,4)
    for cls_label, cls_id in zip([1,2,3], [1,2,4]):  # our network may have output 0-3
        # Create binary mask for this class
        binary_mask = (preds_class == cls_label).astype(np.uint8)
        rle = rle_encode_binary(binary_mask)
        submission.append({
            "id": f"{patient_id}_{cls_id}",
            "rle": rle
        })

# Save CSV
df = pd.DataFrame(submission)
df.to_csv("submission2.csv", index=False)
df.head()

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
5/5 ━━━━━━━━